In [333]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in the data
basicDF = pd.read_csv('Basic_Information.csv', sep = ';')
measurementsDF = pd.read_csv('Body_Measurements.csv', sep = ';', usecols=['Subject', 'Waist girth', 'Hip girth', 'Neck girth' ])
print(measurementsDF.shape)
print(basicDF.shape)


(2011, 4)
(133, 8)


In [334]:
# View full data with scroll bars
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [335]:
# Drop columns Phase, Pantone
basicDF = basicDF.drop(['Phase', 'Pantone'], axis=1)

In [336]:
# Check for missing values
print(basicDF.isnull().sum())
print(measurementsDF.isnull().sum())

# Drop the missing values
basicDF = basicDF.dropna()
measurementsDF = measurementsDF.dropna()


Subject         0
Sex             0
Age (years)     0
Weight (kg)     0
Height (m)      0
BMI (kg/m^2)    0
dtype: int64
Subject         0
Neck girth     10
Waist girth    10
Hip girth      10
dtype: int64


In [337]:
# Print the counts for each Subject if the count is higher than 1
#print(measurementsDF['Subject'].value_counts()[measurementsDF['Subject'].value_counts() > 1])

In [338]:
# It seems like the waist measurements are in mm and for every patient there are two measurements representing a small variance in the measurement. We will take the average of the two measurements for each patient.
measurementsDF = measurementsDF.groupby('Subject').mean()
measurementsDF.head()
measurementsDF.shape


(133, 3)

In [339]:
# Merge the two dataframes
mergedDF = pd.merge(basicDF, measurementsDF, on='Subject')
print(mergedDF.shape)
mergedDF.head()

(133, 9)


,Subject,Sex,Age (years),Weight (kg),Height (m),BMI (kg/m^2),Neck girth,Waist girth,Hip girth
0,IEEEP1_12,female,23,51.1,1.610,19.713746,301.924750,660.366167,937.348167
1,IEEEP1_14,female,19,83.5,1.690,29.235671,377.904417,970.670083,1056.706000
2,IEEEP1_13,female,18,66.6,1.686,23.429288,339.186333,746.001917,1023.511417
3,IEEEP1_15,male,31,82.5,1.778,26.096991,395.601083,921.519417,1041.340750
4,IEEEP1_18,male,26,67.5,1.663,24.407274,380.223833,813.314583,1003.087250


In [340]:
# Simplify the column names
# First remove anything inside parentheses
mergedDF.columns = mergedDF.columns.str.replace(r"\(.*\)","")
# Then replac any spaces with underscores if there is more than one word
mergedDF.columns = mergedDF.columns.str.replace(" ","")
# Make lower case
mergedDF.columns = mergedDF.columns.str.lower()
# Change column waistgirth to waist
mergedDF = mergedDF.rename(columns={'waistgirth':'waist'})
# Change column hipgirth to hip
mergedDF = mergedDF.rename(columns={'hipgirth':'hip'})
# Change column neckgirth to neck
mergedDF = mergedDF.rename(columns={'neckgirth':'neck'})

# Change column sex to gender
mergedDF = mergedDF.rename(columns={'sex':'gender'})

# Change male / female in gender column to 0 / 1
mergedDF = mergedDF.replace('male', 0)
mergedDF = mergedDF.replace('female ', 1)
mergedDF = mergedDF.replace('female', 1)

# Show column names
mergedDF.columns

C:\Users\zrbev\AppData\Local\Temp\ipykernel_10948\769782461.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  mergedDF.columns = mergedDF.columns.str.replace(r"\(.*\)","")


Index(['subject', 'gender', 'age', 'weight', 'height', 'bmi', 'neck', 'waist',
       'hip'],
      dtype='object')

In [341]:
# We need our units to be in KG and CM
# Convert height from m to cm 
mergedDF['height'] = mergedDF['height']*100

# Convert waist from mm to cm 
mergedDF['waist'] = mergedDF['waist']/10
# Convert hip from mm to cm
mergedDF['hip'] = mergedDF['hip']/10
# Convert neck from mm to cm
mergedDF['neck'] = mergedDF['neck']/10

In [342]:
mergedDF.head()

,subject,gender,age,weight,height,bmi,neck,waist,hip
0,IEEEP1_12,1,23,51.1,161.0,19.713746,30.192475,66.036617,93.734817
1,IEEEP1_14,1,19,83.5,169.0,29.235671,37.790442,97.067008,105.670600
2,IEEEP1_13,1,18,66.6,168.6,23.429288,33.918633,74.600192,102.351142
3,IEEEP1_15,0,31,82.5,177.8,26.096991,39.560108,92.151942,104.134075
4,IEEEP1_18,0,26,67.5,166.3,24.407274,38.022383,81.331458,100.308725


In [343]:
# Export the merged dataframe to a csv file
mergedDF.to_csv('mergedDF.csv', index=False)